
# Handling loggers

This script takes you through various ways to handle loggers using MedCAT.

In [ ]:
try:
    from medcat.cat import CAT
except:
    print("WARNING: Runtime will restart automatically and please run other cells thereafter.")
    exit()

In [ ]:
# import medcat
import medcat
# we will use the below later
from medcat import config
from medcat import cat
import os
# print out version string
print(medcat.__version__)
# make sure there is a logger
if not hasattr(medcat, 'logger'):
    raise ValueError("This is an incompatible version!")
print("The package logger:", medcat.logger)

def reset_all_logger_handlers(log_file='temp_medcat.log'): # reset logger handlers in case a block is run multiple times
    medcat.logger.handlers = medcat.logger.handlers[:1] # include the default NullHandler
    config.logger.handlers = []
    cat.logger.handlers = []
    # remove temp log file if it exists
    if os.path.exists(log_file):
        os.remove(log_file)

In [ ]:
log_file = 'temp_medcat.log'
import os
# the default behaviour is to not log anything, the following should thus not create any output
medcat.logger.warning("This should be ignored")
print('Log file should not exists, does it?', os.path.exists(log_file))
# enable default loggging to the package-level logger
medcat.add_default_log_handlers(target_file=log_file)
# now we should have a console logger as well as a log in medcat.log
# so we should see the following output to both
msg = "This message should show up in console and be written to medcat.log"
medcat.logger.warning(msg)
with open(log_file, 'r') as f:
    last_line = f.readlines()[-1][:-1] # last line, ignoring the newline char
    print("Last line should be equal to our message", msg == last_line)
reset_all_logger_handlers(log_file) # for cleanup

In [ ]:
import logging
config.logger.addHandler(logging.StreamHandler())
# now, the medcat.logger won't log into console
medcat.logger.error("This error does not show up")
# however, the config.logger will
config.logger.warning("This warning will show up")
# and at the same time, we can see that the logger of cat won't log anything either
cat.logger.warning("This warning will not show up either")
reset_all_logger_handlers() # for cleanup

In [ ]:
target_file_config = 'medcat_config.log' # some target log file for config logger
target_file_cat = 'medcat_cat.log' # different log file for cat
# set up different file handlers for the two different modules
config.logger.addHandler(logging.FileHandler(target_file_config))
cat.logger.addHandler(logging.FileHandler(target_file_cat))
# config now logs into a different file than cat
# i.e the following gets logged into config's log file
config.logger.warning("There has been an issue with the config")
# and the following gets logged into cat's log file
cat.logger.error("There was a critical issue in CAT")
# we can check that by looking at the files
with open(target_file_config) as f:
    config_contents = f.read()
with open(target_file_cat) as f:
    cat_contents = f.read()
print('Config log contents:\n', config_contents)
print('CAT log contents:\n', cat_contents)
# cleanup
reset_all_logger_handlers(target_file_config)
reset_all_logger_handlers(target_file_cat)